# Creating and Valuing a CDS Contract

Creates a CDS contract and does a valuation and calculates risk measures

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from financepy.utils import *
from financepy.products.rates import *
from financepy.products.credit import *

## Creating a CDS Contract

In [3]:
value_dt = Date(10, 1, 2026)
effective_dt = Date(1,12,2024)
maturity_dt = Date(20, 3, 2028)
cds_coupon = 0.010
notional = ONE_MILLION
long_protection = False
trade_dt = Date(9, 8, 2019)

In [4]:
cds_contract = CDS(effective_dt, maturity_dt, cds_coupon, notional, long_protection)

In [48]:
print(cds_contract)

OBJECT TYPE: CDS
STEP-IN DATE: 01-DEC-2024
MATURITY: 20-MAR-2028
NOTIONAL: 1000000
RUN COUPON: 100.0bp
DAYCOUNT: DayCountTypes.ACT_360
FREQUENCY: FrequencyTypes.QUARTERLY
CALENDAR: CalendarTypes.WEEKEND
BUSDAYRULE: BusDayAdjustTypes.FOLLOWING
DATEGENRULE: DateGenRuleTypes.BACKWARD
PAYMENT_dt, YEAR_FRAC, ACCRUAL_START, ACCRUAL_END, FLOW
20-DEC-2024,     0.252778, 20-SEP-2024, 19-DEC-2024,  2527.777778
20-MAR-2025,     0.250000, 20-DEC-2024, 19-MAR-2025,  2500.000000
20-JUN-2025,     0.255556, 20-MAR-2025, 19-JUN-2025,  2555.555556
22-SEP-2025,     0.261111, 20-JUN-2025, 21-SEP-2025,  2611.111111
22-DEC-2025,     0.252778, 22-SEP-2025, 21-DEC-2025,  2527.777778
20-MAR-2026,     0.244444, 22-DEC-2025, 19-MAR-2026,  2444.444444
22-JUN-2026,     0.261111, 20-MAR-2026, 21-JUN-2026,  2611.111111
21-SEP-2026,     0.252778, 22-JUN-2026, 20-SEP-2026,  2527.777778
21-DEC-2026,     0.252778, 21-SEP-2026, 20-DEC-2026,  2527.777778
22-MAR-2027,     0.252778, 21-DEC-2026, 21-MAR-2027,  2527.777778
21

## Build Discount Curve

In [6]:
value_dt = Date(1, 1, 2026)
interest_rate = 0.03

In [7]:
from financepy.market.curves import DiscountCurveFlat

In [8]:
discount_curve = DiscountCurveFlat(value_dt, interest_rate, FrequencyTypes.CONTINUOUS)

In [9]:
print(discount_curve)

OBJECT TYPE: DiscountCurveFlat
VALUE DATE: 01-JAN-2026
FLAT RATE: 0.03
FREQUENCY: FrequencyTypes.CONTINUOUS
DAY COUNT: DayCountTypes.ACT_ACT_ISDA



# Build a CDS Curve

In [10]:
settle_dt = value_dt.add_days(1)

In [41]:
cds1 = CDS(settle_dt, "1Y", 0.0065)
cds2 = CDS(settle_dt, "2Y", 0.0070)
cds3 = CDS(settle_dt, "3Y", 0.0075)
cds5 = CDS(settle_dt, "5Y", 0.0080)

In [42]:
print(cds2)

OBJECT TYPE: CDS
STEP-IN DATE: 02-JAN-2026
MATURITY: 20-MAR-2028
NOTIONAL: 1000000
RUN COUPON: 70.0bp
DAYCOUNT: DayCountTypes.ACT_360
FREQUENCY: FrequencyTypes.QUARTERLY
CALENDAR: CalendarTypes.WEEKEND
BUSDAYRULE: BusDayAdjustTypes.FOLLOWING
DATEGENRULE: DateGenRuleTypes.BACKWARD
PAYMENT_dt, YEAR_FRAC, ACCRUAL_START, ACCRUAL_END, FLOW
20-MAR-2026,     0.244444, 22-DEC-2025, 19-MAR-2026,  1711.111111
22-JUN-2026,     0.261111, 20-MAR-2026, 21-JUN-2026,  1827.777778
21-SEP-2026,     0.252778, 22-JUN-2026, 20-SEP-2026,  1769.444444
21-DEC-2026,     0.252778, 21-SEP-2026, 20-DEC-2026,  1769.444444
22-MAR-2027,     0.252778, 21-DEC-2026, 21-MAR-2027,  1769.444444
21-JUN-2027,     0.252778, 22-MAR-2027, 20-JUN-2027,  1769.444444
20-SEP-2027,     0.252778, 21-JUN-2027, 19-SEP-2027,  1769.444444
20-DEC-2027,     0.252778, 20-SEP-2027, 19-DEC-2027,  1769.444444
20-MAR-2028,     0.255556, 20-DEC-2027, 20-MAR-2028,  1788.888889


In [43]:
cds_list = [cds1, cds2, cds3, cds5]

In [44]:
recovery_rate = 0.40

In [45]:
cds_curve = CDSCurve(value_dt, cds_list, discount_curve, recovery_rate)

In [46]:
print(cds_curve)

OBJECT TYPE: CDSCurve
TIME,SURVIVAL_PROBABILITY
 0.0000000,  1.0000000
 1.2136986,  0.9865973
 2.2164384,  0.9739514
 3.2164384,  0.9597968
 5.2164384,  0.9314422


# Valuation

In [17]:
cds_contract.print_payments(value_dt, cds_curve)

PAYMENT_dt      YEAR_FRAC      FLOW           DF       SURV_PROB      NPV
    20-MAR-2026   0.244444      2444.44     0.993610     0.997627      2423.06
    22-JUN-2026   0.261111      2611.11     0.985962     0.994775      2561.01
    21-SEP-2026   0.252778      2527.78     0.978616     0.992021      2453.99
    21-DEC-2026   0.252778      2527.78     0.971323     0.989275      2428.96
    22-MAR-2027   0.252778      2527.78     0.964085     0.986528      2404.16
    21-JUN-2027   0.252778      2527.78     0.956902     0.983368      2378.61
    20-SEP-2027   0.252778      2527.78     0.949771     0.980219      2353.32
    20-DEC-2027   0.252778      2527.78     0.942694     0.977080      2328.30
    20-MAR-2028   0.255556      2555.56     0.935669     0.973951      2328.87


In [51]:
cds_contract.risky_pv01(value_dt, cds_curve)


{'dirty_rpv01': np.float64(2.1857820395029024),
 'clean_rpv01': np.float64(2.1580042617251247)}

In [50]:
cds2.par_spread(value_dt, cds_curve, recovery_rate) * 10000.0


np.float64(69.99999992944535)

In [49]:
spd = cds_contract.par_spread(settle_dt, cds_curve, recovery_rate) * 10000.0
print("FAIR CDS SPREAD %10.5f bp"% spd)

FAIR CDS SPREAD   69.98721 bp


In [24]:
v = cds_contract.value(settle_dt, cds_curve, recovery_rate)

In [25]:
dirty_pv = v['dirty_pv']
clean_pv = v['clean_pv']

In [26]:
print("DIRTY VALUE %12.2f"% dirty_pv)
print("CLEAN VALUE %12.2f"% clean_pv)

DIRTY VALUE      6774.74
CLEAN VALUE      6469.19


In [27]:
cleanp = cds_contract.clean_price(settle_dt, cds_curve, recovery_rate)
print("CLEAN PRICE %12.6f"% cleanp)

CLEAN PRICE   100.646919


In [28]:
accrued_days = cds_contract.accrued_days(value_dt)
print("ACCRUED_DAYS", accrued_days)

ACCRUED_DAYS 10.0


In [29]:
accrued_interest = cds_contract.accrued_interest(value_dt)
print("ACCRUED_COUPON", accrued_interest)

ACCRUED_COUPON 277.77777777777777


In [30]:
prot_pv = cds_contract.prot_leg_pv(settle_dt, cds_curve, recovery_rate)
print("prot_PV", prot_pv)

prot_PV 15085.579368313174


In [31]:
premPV = cds_contract.premium_leg_pv(settle_dt, cds_curve, recovery_rate)
print("PREMIUM_PV", premPV)

PREMIUM_PV 21860.322230598802


In [32]:
rpv01 = cds_contract.risky_pv01(settle_dt, cds_curve)
print("DIRTY_RPV01", rpv01['dirty_rpv01'])
print("CLEAN_RPV01", rpv01['clean_rpv01'])

DIRTY_RPV01 2.18603222305988
CLEAN_RPV01 2.1554766675043244


In [33]:
cds_contract.risky_pv01(settle_dt, cds_curve)

{'dirty_rpv01': np.float64(2.18603222305988),
 'clean_rpv01': np.float64(2.1554766675043244)}

In [54]:
cds_contract.value_fast_approx(value_dt, interest_rate, 0.070, recovery_rate)

(np.float64(-112993.89169750002),
 np.float64(-113271.6694752778),
 np.float64(-724.9588986598683),
 np.float64(-543.682345703477))

## Risk Measures

In [34]:
credit_dv01 = cds_contract.credit_dv01(settle_dt, cds_curve, recovery_rate)

In [35]:
credit_dv01

np.float64(-216.70636200452282)

Copyright (c) 2020 Dominic O'Kane